## CIFAR CONV


[Explain about data]
[Explain about convolutional]

In [1]:
import os
import tensorflow as tf
import numpy as np
from mlp.data_providers import CIFAR10DataProvider, CIFAR100DataProvider
import matplotlib.pyplot as plt
import pdb

In [2]:
def _variable_with_weight_decay(name, shape, stddev, wd):
  """Helper to create an initialized Variable with weight decay.
  Note that the Variable is initialized with a truncated normal distribution.
  A weight decay is added only if one is specified.
  Args:
    name: name of the variable
    shape: list of ints
    stddev: standard deviation of a truncated Gaussian
    wd: add L2Loss weight decay multiplied by this float. If None, weight
        decay is not added for this Variable.
  Returns:
    Variable Tensor
  """
  dtype = tf.float32
  var =  tf.Variable(
        tf.truncated_normal(
            shape), 
        'weights')

  if wd is not None:
    weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
    tf.add_to_collection('losses', weight_decay)
  return var

In [7]:
num_hidden = 200
num_hidden2 = 100
num_hidden3 = 50
BATCH_SIZE = 50
NUM_CLASSES = 10
input_dim = 32
output_dim = 32
EPOCH_SIZE = 100

In [ ]:
train_data = CIFAR10DataProvider('train', batch_size=BATCH_SIZE)
train_data.inputs = train_data.inputs.reshape((-1, 32, 32, 3))
valid_data = CIFAR10DataProvider('valid', batch_size=BATCH_SIZE)
valid_data.inputs = valid_data.inputs.reshape((-1, 32, 32, 3))

In [4]:
# place holder for input and target
inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1], train_data.inputs.shape[2], train_data.inputs.shape[3]], 'inputs')
targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')

First of all we try to build a simple convolutional Neural network as a baseline of our next model.
this network consist of one convolutional layer - one dense layer

In [5]:
# building graph

conv1_out_size = 14 #number of output channel of first convolutional 
with tf.name_scope('conv-1') as scope:
    kernel = _variable_with_weight_decay('weights',
                                         shape=[5, 5, 3, conv1_out_size],
                                         stddev=5e-2,
                                         wd=0.0)

    conv = tf.nn.conv2d(inputs, kernel, [1, 1, 1, 1], padding='SAME')
    #biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
    biases = tf.Variable(tf.zeros([conv1_out_size]), 'biases') 
    pre_activation = tf.nn.bias_add(conv, biases)
    # conv1 = tf.nn.relu(pre_activation)
    local1 = tf.nn.relu(pre_activation)
    # pool1
    pool1 = tf.nn.max_pool(local1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='SAME')

# Move everything into depth so we can perform a single matrix multiply.
with tf.name_scope('Dense-Relu_Layer') as scope:
    # flattening the input
    tot_shape=pool1.get_shape()[1].value*pool1.get_shape()[2].value*pool1.get_shape()[3].value
    reshape = tf.reshape(pool1, [BATCH_SIZE,tot_shape])
    weights = _variable_with_weight_decay('weights3', shape=[tot_shape, 384],stddev=1.0, wd=0.0)
    biases = tf.Variable(tf.zeros([384]), 'biases') 
    local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases)


with tf.variable_scope('softmax_linear') as scope:
    weights = _variable_with_weight_decay('weights5', [384, NUM_CLASSES],
                                          stddev=1.0, wd=0.0)
    biases = tf.Variable(tf.zeros([NUM_CLASSES]), 'biases') 
    softmax_linear = tf.add(tf.matmul(local3, weights), biases)
    soft_max_out = tf.nn.softmax(softmax_linear)

with tf.name_scope('error'):
    out_login = tf.nn.softmax_cross_entropy_with_logits(logits = softmax_linear, labels=targets)
    error = tf.reduce_mean(out_login)

# use softmax for accuracy
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(soft_max_out, 1), tf.argmax(targets, 1)), 
            tf.float32))f

# use adam optimizer 
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer().minimize(error)
    
init = tf.global_variables_initializer()

In [ ]:
# dictionary to save all the data
all_train_score_d = {}
all_valid_score_d = {}

In [8]:
# begin training
experiment_name = "(1) simple conv-net"
acc_train_list = []
err_train_list = []
acc_valids = []
err_valids = []
with tf.Session() as sess:
    sess.run(init)
    for e in range(EPOCH_SIZE):
        running_error = 0.
        running_accuracy = 0.
        for input_batch, target_batch in train_data:            
            _, batch_error, batch_acc = sess.run(
                [train_step, error, accuracy], 
                feed_dict={inputs: input_batch, targets: target_batch})
            # calculating error and accuracy for batch
            running_error += batch_error
            running_accuracy += batch_acc
        # averaging the error and accuracy
        running_error /= train_data.num_batches
        running_accuracy /= train_data.num_batches
        print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f}'
              .format(e + 1, running_error, running_accuracy))
        acc_train_list.append(running_accuracy)
        err_train_list.append(running_error)

        # validation
        if  (e + 1) % 1 == 0:
            valid_error = 0.
            valid_accuracy = 0.
            for input_batch, target_batch in valid_data:
                batch_error, batch_acc = sess.run(
                    [error, accuracy], 
                    feed_dict={inputs: input_batch, targets: target_batch})
                valid_error += batch_error
                valid_accuracy += batch_acc
            valid_error /= valid_data.num_batches
            valid_accuracy /= valid_data.num_batches
            print('                 err(valid)={0:.2f} acc(valid)={1:.2f}'
                   .format(valid_error, valid_accuracy))
            acc_valids.append(valid_accuracy)
            err_valids.append(valid_error)
all_train_score_d[experiment_name] = [acc_train_list,err_train_list]
all_valid_score_d[experiment_name] = [acc_valids,err_valids]

End of epoch 01: err(train)=291.29 acc(train)=0.18
                 err(valid)=259.70 acc(valid)=0.27
End of epoch 02: err(train)=204.64 acc(train)=0.29
                 err(valid)=171.76 acc(valid)=0.29
End of epoch 03: err(train)=137.45 acc(train)=0.31
                 err(valid)=120.11 acc(valid)=0.30
End of epoch 04: err(train)=98.46 acc(train)=0.31
                 err(valid)=89.21 acc(valid)=0.30
End of epoch 05: err(train)=73.11 acc(train)=0.32
                 err(valid)=66.85 acc(valid)=0.28
End of epoch 06: err(train)=49.16 acc(train)=0.29
                 err(valid)=37.47 acc(valid)=0.25
End of epoch 07: err(train)=25.50 acc(train)=0.24
                 err(valid)=19.48 acc(valid)=0.21
End of epoch 08: err(train)=12.94 acc(train)=0.19
                 err(valid)=10.39 acc(valid)=0.17
End of epoch 09: err(train)=8.22 acc(train)=0.18
                 err(valid)=7.33 acc(valid)=0.18
End of epoch 10: err(train)=5.92 acc(train)=0.19
                 err(valid)=5.54 acc(valid)=0.1

KeyboardInterrupt: 

In [ ]:
single_plot_graph(err_train_list,acc_train_list)

In [ ]:
def single_plot_graph(err_list,acc_list,scale=None):
    """
        
    """

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(211)
    if not scale:
        ax_1.plot(acc_list,label='accuration list')
    else:
        ax_1.plot(np.arange(1,100,scale),acc_list,label='accuration list')
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    
    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(212)
    if not scale:
        ax_2.plot(err_list,label='error list',c='r')
    else:
        ax_2.plot(np.arange(1,120,scale),err_list,label='error list',c='r')
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    

In [ ]:
def multi_plot_result(filelist,valid=False,_title=None):
    result_dict = {}
    fig_1 = plt.figure(figsize=(8, 4))
    scale = 5
    if _title:
        print(_title)
        plt.title(_title)
    for filetup in filelist:
        filename,label=filetup
        filepath= "/afs/inf.ed.ac.uk/user/s15/s1575408/mlpractical/script/"+filename+".txt"
        result_dict[filename]=open_file(filepath)
        if valid:
            plt.plot(np.arange(4,121,scale),result_dict[filename],label=label)
        else:
            plt.plot(result_dict[filename],label=label)
        lgd=plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.savefig(_title+".svg",bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
def pltsin(ax, colors=['b']):
    x = np.linspace(0,1,100)
    if ax.lines:
        for line in ax.lines:
            line.set_xdata(x)
            y = np.random.random(size=(100,1))
            line.set_ydata(y)
    else:
        for color in colors:
            y = np.random.random(size=(100,1))
            ax.plot(x, y, color)
    fig.canvas.draw()

In [ ]:
filelist =  [("acc.relu_50.train","relu-50 neurons"),("acc.relu_500.train","relu-500 neurons")
,("acc_trains_model1","softmax-1024 neurons"),("acc_trains_model2","relu-1024 neurons"),
             ("acc_trains_model3","relu-200 neurons"),("acc_trains_model4","tanh-200 neurons")]
plot_result(filelist, _title='train accuracy experiment 1')
filelist = [("accrelu_50.valid","relu-50 neurons"),("accrelu_500.valid","relu-500 neurons")
,("acc_valid_model1","softmax-1024 neurons"),("acc_valid_model2","relu-1024 neurons"),
            ("acc_valid_model3","relu-200 neurons"),("acc_valid_model4","tanh-200 neurons")]
plot_result(filelist,valid=True,_title="validation accuracy experiment 1")

filelist =  [("err.relu_50.train","relu-50 neurons"),("err.relu_500.train","relu-500 neurons"),("error_trains_model1","softmax-1024 neurons"),("error_trains_model2","relu-1024 neurons"),
             ("error_trains_model3","relu-200 neurons"),("error_trains_model4","tanh-200 neurons")]
plot_result(filelist, _title='train error experiment 1')
filelist = [("err.relu_50.valid","relu-50 neurons"),("err.relu_500.valid","relu-500 neurons"),("error_valid_model1","softmax-1024 neurons"),("error_valid_model2","relu-1024 neurons"),
            ("error_valid_model3","relu-200 neurons"),("error_valid_model4","tanh-200 neurons")]
plot_result(filelist,valid=True,_title='validation error experiment 1')
